In [2]:
#https://dylancastillo.co/text-classification-using-python-and-scikit-learn/
#import packages
import pandas as pd
import numpy as np
import re
import string
import os
import emoji
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.feature_selection import chi2,SelectKBest
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline, make_union
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
import joblib

In [3]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings('ignore')

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import *
from xgboost import XGBClassifier

In [5]:
df = pd.read_csv (r'.\data\Womens Clothing E-Commerce Reviews.csv').drop(columns = 'Unnamed: 0')
df.head()

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [6]:
df.shape

(23486, 10)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486 entries, 0 to 23485
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Clothing ID              23486 non-null  int64 
 1   Age                      23486 non-null  int64 
 2   Title                    19676 non-null  object
 3   Review Text              22641 non-null  object
 4   Rating                   23486 non-null  int64 
 5   Recommended IND          23486 non-null  int64 
 6   Positive Feedback Count  23486 non-null  int64 
 7   Division Name            23472 non-null  object
 8   Department Name          23472 non-null  object
 9   Class Name               23472 non-null  object
dtypes: int64(5), object(5)
memory usage: 1.8+ MB


In [8]:
df['Recommended IND'].value_counts()

1    19314
0     4172
Name: Recommended IND, dtype: int64

In [9]:
#undersample 
df1_undersample = df[df['Recommended IND']==1].sample(4172, random_state = 42)
df_undersample  = pd.concat([df1_undersample, df[df['Recommended IND']==0]])
df_undersample.head()


,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
14984,831,60,"Very pretty, too blousy","This is a very pretty blouse with a vintage, r...",5,1,1,General Petite,Tops,Blouses
8633,828,60,Nice staple with a twist,This is a really nice white button-up top. per...,5,1,3,General,Tops,Blouses
5366,877,25,Love love,Love this shirt and the print. i would wear it...,5,1,0,General,Tops,Knits
12124,1126,36,Lovely fall coat,I just received this coat and love it. the onl...,4,1,0,General,Jackets,Outerwear
20209,685,26,Sexy and comfy,This robe is exactly what i was looking for. ...,5,1,0,Initmates,Intimate,Sleep


In [10]:
df_undersample.shape

(8344, 10)

In [11]:
#some data is missing  Title,  review text, division to class 

num_column = ['Age',  'Rating', 'Positive Feedback Count' ]
cat_column = ['Division Name', 'Department Name', 'Class Name']
txt_column = [ 'Title', 'Review Text'] 


In [12]:
txt_column

['Title', 'Review Text']

In [13]:
X =  df_undersample.drop(['Clothing ID','Recommended IND'], axis =1)
y =  df_undersample['Recommended IND']
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)

In [14]:
df[num_column ].describe()

,Age,Rating,Positive Feedback Count
count,23486.000000,23486.000000,23486.000000
mean,43.198544,4.196032,2.535936
std,12.279544,1.110031,5.702202
min,18.000000,1.000000,0.000000
25%,34.000000,4.000000,0.000000
50%,41.000000,5.000000,1.000000
75%,52.000000,5.000000,3.000000
max,99.000000,5.000000,122.000000


In [15]:
df['Positive Feedback Count'].apply(lambda x : np.log(x+1)).min()

0.0

In [16]:
#log cannot hand negtive or 0, instead using some 
def logTransformer (df):
    return df.apply(lambda x : np.log(x+1))


log_transformer = FunctionTransformer(logTransformer)

In [17]:
X[num_column]

,Age,Rating,Positive Feedback Count
14984,60,5,1
8633,60,5,3
5366,25,5,0
12124,36,4,0
20209,26,5,0
...,...,...,...
23449,39,3,7
23450,40,3,4
23460,34,3,0
23478,32,1,0


In [18]:
x1 = log_transformer.transform(X[num_column])
x1
# x2= 
# x2

,Age,Rating,Positive Feedback Count
14984,4.110874,1.791759,0.693147
8633,4.110874,1.791759,1.386294
5366,3.258097,1.791759,0.000000
12124,3.610918,1.609438,0.000000
20209,3.295837,1.791759,0.000000
...,...,...,...
23449,3.688879,1.386294,2.079442
23450,3.713572,1.386294,1.609438
23460,3.555348,1.386294,0.000000
23478,3.496508,0.693147,0.000000


In [19]:
#make sure number feature can be  preprocessed
num_pipeline = make_pipeline(log_transformer, SimpleImputer(strategy='median'), StandardScaler())
n= num_pipeline.fit_transform(X_train[num_column])
n.shape

(6675, 3)

In [20]:
cat_pipeline = make_pipeline(SimpleImputer(strategy='constant', fill_value='Missing'), OneHotEncoder(handle_unknown='ignore'))
cat_pipeline.fit_transform(X_train[cat_column])

<6675x30 sparse matrix of type '<class 'numpy.float64'>'
	with 20025 stored elements in Compressed Sparse Row format>

In [21]:
#pipeline handle text 


In [22]:
#https://towardsdatascience.com/sentiment-analysis-with-text-mining-13dd2b33de27
class CleanText(BaseEstimator, TransformerMixin):
    def remove_mentions(self, input_text):
        return re.sub(r'@\w+', '', str(input_text))
    
    def remove_urls(self, input_text):
        return re.sub(r'http.?://[^\s]+[\s]?', '', str(input_text))
    
    def emoji_oneword(self, input_text):
        # By compressing the underscore, the emoji is kept as one word
        return str(input_text).replace('_','')
    
    def remove_punctuation(self, input_text):
        # Make translation table
        punct = string.punctuation
        trantab = str.maketrans(punct, len(punct)*' ')  # Every punctuation symbol will be replaced by a space
        return str(input_text).translate(trantab)
    def remove_digits(self, input_text):
        return re.sub('\d+', '', str(input_text))
    
    def to_lower(self, input_text):
        return str(input_text).lower()
    
    def remove_stopwords(self, input_text):
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = str(input_text).split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words) 
    
    def stemming(self, input_text):
        porter = PorterStemmer()
        words = str(input_text).split() 
        stemmed_words = [porter.stem(word) for word in words]
        return " ".join(stemmed_words)
    
#     def fill_blank_text (self, input_text):
#         empty_clean = input_text == ''
#         #print('{} records have no words left after text cleaning'.format(sr_clean[empty_clean].count()))
#         input_text.loc[empty_clean] = '[no_text]'
    
    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X, **transform_params):
        clean_X = X.apply(self.remove_mentions).apply(self.remove_urls).apply(self.emoji_oneword).apply(self.remove_punctuation).apply(self.remove_digits).apply(self.to_lower).apply(self.remove_stopwords).apply(self.stemming)
        return clean_X

In [27]:
mystopwords = (stopwords.words())
top_k_features = 100

In [28]:
txt1_pipeline =  make_pipeline ( SimpleImputer(strategy='constant', fill_value='Missing'),                #title   
                    CountVectorizer(stop_words=mystopwords,lowercase=True),
                    SelectKBest(chi2,k=20), 
                    TfidfTransformer(use_idf=True))
txt2_pipeline =  make_pipeline (SimpleImputer(strategy='constant', fill_value='Missing'),   
                    CleanText(),                                                                 
                    CountVectorizer(stop_words=mystopwords,lowercase=True),
                    SelectKBest(chi2,k=top_k_features),  #this need y train 
                    TfidfTransformer(use_idf=True))


In [29]:
x1 = SimpleImputer(strategy='constant', fill_value='Missing').fit_transform(X_train['Title'])
x1

ValueError: Expected 2D array, got 1D array instead:
array=['More compliments than i could have imagined' 'Perfect fit'
 'Perfectly drapey' ... 'Not for me... and not at this price'
 'Charming and basic' 'Black bleeds on cream!'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [30]:
x1 = SimpleImputer(strategy='constant', fill_value='Missing').fit_transform(np.array(X_train['Title']).reshape(-1, 1)).ravel()
x1

array(['More compliments than i could have imagined', 'Perfect fit',
       'Perfectly drapey', ..., 'Not for me... and not at this price',
       'Charming and basic', 'Black bleeds on cream!'], dtype=object)

In [31]:
#to make it easier, create a custom class
#https://stackoverflow.com/questions/63000388/how-to-include-simpleimputer-before-countvectorizer-in-a-scikit-learn-pipeline
class OneDWrapper:
    """One dimensional wrapper for sklearn Transformers"""

    def __init__(self, transformer):
        self.transformer = transformer

    def fit(self, X, y=None):
        self.transformer.fit(np.array(X).reshape(-1, 1))
        return self

    def transform(self, X, y=None):
        return self.transformer.transform(
            np.array(X).reshape(-1, 1)).ravel()

    def inverse_transform(self, X, y=None):
        return self.transformer.inverse_transform(
            np.expand_dims(X, axis=1)).ravel()

In [32]:
one_d_imputer = OneDWrapper(SimpleImputer(strategy='constant', fill_value='Missing'))
txt1_pipeline =  make_pipeline ( one_d_imputer,                
                    CountVectorizer(stop_words=mystopwords,lowercase=True),
                    SelectKBest(chi2,k=20), 
                    TfidfTransformer(use_idf=True))
a=txt1_pipeline.fit_transform(X_train['Title'], y_train) 
a

<6675x20 sparse matrix of type '<class 'numpy.float64'>'
	with 2541 stored elements in Compressed Sparse Row format>

In [33]:
CleanText().fit_transform(X_train['Review Text'])

9057     knew bought bralett someth special intricaci d...
21033    beauti top run tt armhol not big typic struggl...
23472    saw shirt retail websit new arriv perfectli dr...
2426     realli want keep dress materi amaz soft gentl ...
4384     dress look feel cheap white materi stiff not i...
                               ...                        
8977     jacket cute fit perfect love motif howev depos...
5745     top made well strap holder keep bra hidden alw...
7163     thought top would nice addit fall wardrob colo...
23114    got tan one friend got skirt tini not hip got ...
17473    top realli love high hope look smash black jea...
Name: Review Text, Length: 6675, dtype: object

In [34]:
txt2_pipeline =  make_pipeline (CleanText(),
                    one_d_imputer,                                                                                       
                    CountVectorizer(stop_words=mystopwords,lowercase=True),
                    SelectKBest(chi2,k=top_k_features),  #this need y train 
                    TfidfTransformer(use_idf=True))
b=txt2_pipeline.fit_transform(X_train['Review Text'], y_train) 
b

<6675x100 sparse matrix of type '<class 'numpy.float64'>'
	with 44108 stored elements in Compressed Sparse Row format>

In [35]:
preprocessor = make_column_transformer (
    (num_pipeline, num_column ),
    (cat_pipeline, cat_column),
    (txt1_pipeline, ['Title']),
    (txt2_pipeline, ['Review Text'])
)
preprocessor

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('functiontransformer',
                                                  FunctionTransformer(func=<function logTransformer at 0x00000232BEDAB8B0>)),
                                                 ('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['Age', 'Rating', 'Positive Feedback Count']),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(f...
                                                                              'إذما',
                                                                              'إذن',
                             

In [36]:
#https://stackoverflow.com/questions/66657086/sklearn-found-input-variables-with-inconsistent-numbers-of-samples
preprocessor = make_column_transformer (
    (num_pipeline, num_column ),
     (cat_pipeline, cat_column),
     (txt1_pipeline, ['Title']),
     (txt2_pipeline, 'Review Text')
)

preprocessor.fit_transform(X_train, y_train) 


<6675x153 sparse matrix of type '<class 'numpy.float64'>'
	with 86699 stored elements in Compressed Sparse Row format>

In [37]:
# Initialze the estimators
clf1 = RandomForestClassifier(random_state=42, )
clf2 = SVC(probability=True, random_state=42)
clf3 = LogisticRegression(random_state=42, penalty='l1', solver='liblinear')
clf4 = DecisionTreeClassifier(random_state=42)
clf5 = KNeighborsClassifier()
clf6 = MultinomialNB()
clf7 = GradientBoostingClassifier(random_state=42)
clf8 =  XGBClassifier(random_state=42, eval_metric='logloss')

In [38]:
# Initiaze the hyperparameters for each dictionary
param1 = {}
param1['classifier__n_estimators'] = [10, 50, 100, 250]
param1['classifier__max_depth'] = [5, 10, 20]
param1['classifier__class_weight'] = [None, {0:1,1:5}, {0:1,1:10}, {0:1,1:25}]
param1['classifier'] = [clf1]

param2 = {}
param2['classifier__C'] = [10**-2, 10**-1, 10**0, 10**1, 10**2]
param2['classifier__class_weight'] = [None, {0:1,1:5}, {0:1,1:10}, {0:1,1:25}]
param2['classifier'] = [clf2]

param3 = {}
param3['classifier__C'] = [10**-2, 10**-1, 10**0, 10**1, 10**2]
#param3['classifier__penalty'] = ['l1', 'l2']
param3['classifier__class_weight'] = [None, {0:1,1:5}, {0:1,1:10}, {0:1,1:25}]
param3['classifier'] = [clf3]

param4 = {}
param4['classifier__max_depth'] = [5,10,25,None]
param4['classifier__min_samples_split'] = [2,5,10]
param4['classifier__class_weight'] = [None, {0:1,1:5}, {0:1,1:10}, {0:1,1:25}]
param4['classifier'] = [clf4]

param5 = {}
param5['classifier__n_neighbors'] = [2,5,10,25,50]
param5['classifier'] = [clf5]

param6 = {}
param6['classifier__alpha'] = [10**0, 10**1, 10**2]
param6['classifier'] = [clf6]

param7 = {}
param7['classifier__n_estimators'] = [10, 50, 100, 250]
param7['classifier__max_depth'] = [5, 10, 20]
param7['classifier'] = [clf7]

param8 = {}
param8['classifier__n_estimators'] = [100, 400, 800]
param8['classifier__max_depth'] = [3, 6, 9]
param8['classifier__learning_rate'] = [0.05, 0.1, 0.20]
param8['classifier__min_child_weight'] = [1, 10, 100]
param8['classifier'] = [clf8]

In [39]:
pipeline = Pipeline([
     ('preprocessor', preprocessor ) ,  
    ('classifier', clf1)])
params = [param1, param2, param3, param4, param5, param6, param7, param8]

In [40]:
%%time
# Train the grid search model
gs = RandomizedSearchCV(pipeline, params, cv=3, n_iter = 100,  scoring='roc_auc').fit(X_train, y_train)
#n_jobs=-1,

Wall time: 41min 53s


In [41]:
gs.best_params_

{'classifier__class_weight': None,
 'classifier__C': 1,
 'classifier': LogisticRegression(C=1, penalty='l1', random_state=42, solver='liblinear')}

In [46]:
gs_result= pd.concat([pd.DataFrame(gs.cv_results_["params"]),pd.DataFrame(gs.cv_results_["mean_test_score"], columns=["Accuracy"])],axis=1)
gs_result.sort_values(by=['Accuracy'], ascending=False)

,classifier__n_estimators,classifier__min_child_weight,classifier__max_depth,classifier__learning_rate,classifier,classifier__min_samples_split,classifier__class_weight,classifier__C,classifier__n_neighbors,classifier__alpha,Accuracy
18,NaN,NaN,NaN,NaN,"LogisticRegression(C=1, penalty='l1', random_s...",NaN,None,1.0,NaN,NaN,0.977767
13,100.0,10.0,3.0,0.10,"XGBClassifier(base_score=None, booster=None, c...",NaN,NaN,NaN,NaN,NaN,0.976506
65,NaN,NaN,NaN,NaN,"SVC(probability=True, random_state=42)",NaN,"{0: 1, 1: 5}",0.1,NaN,NaN,0.976412
4,100.0,1.0,3.0,0.10,"XGBClassifier(base_score=None, booster=None, c...",NaN,NaN,NaN,NaN,NaN,0.976286
31,100.0,10.0,9.0,0.05,"XGBClassifier(base_score=None, booster=None, c...",NaN,NaN,NaN,NaN,NaN,0.976229
...,...,...,...,...,...,...,...,...,...,...,...
75,NaN,NaN,NaN,NaN,DecisionTreeClassifier(random_state=42),5.0,"{0: 1, 1: 5}",NaN,NaN,NaN,0.912465
1,NaN,NaN,NaN,NaN,DecisionTreeClassifier(random_state=42),5.0,"{0: 1, 1: 25}",NaN,NaN,NaN,0.908240
70,NaN,NaN,NaN,NaN,DecisionTreeClassifier(random_state=42),2.0,"{0: 1, 1: 25}",NaN,NaN,NaN,0.900367
50,NaN,NaN,NaN,NaN,MultinomialNB(),NaN,NaN,NaN,NaN,1.0,NaN


In [52]:
gs.best_score_

0.977767099587853

In [50]:
y_test_hat = gs.predict(X_test)

In [51]:
print(classification_report(y_test, y_test_hat))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94       832
           1       0.95      0.93      0.94       837

    accuracy                           0.94      1669
   macro avg       0.94      0.94      0.94      1669
weighted avg       0.94      0.94      0.94      1669



In [52]:
confusion_matrix(y_test, y_test_hat)

array([[791,  41],
       [ 59, 778]], dtype=int64)

In [49]:
joblib.dump(gs, r'./pickle/handle_different_type.pickle')

['./pickle/handle_different_type.pickle']

In [ ]:
'''
https://github.com/AFAgarap/ecommerce-reviews-analysis

--smote
https://www.linkedin.com/pulse/analysis-womens-e-commerce-clothing-reviews-yun-xiao/

imbalance sampling
https://www.youtube.com/watch?v=YMPMZmlH5Bo&t=602s
https://www.youtube.com/watch?v=OJedgzdipC0
'''

In [24]:
#https://towardsdatascience.com/how-to-combine-textual-and-numerical-features-for-machine-learning-in-python-dc1526ca94d9